<a href="https://colab.research.google.com/github/wicksonkold/Market-Sentiment-Analysis/blob/main/TA_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Money Flow Index
#Relative Strength Index
#Moving Average Convergence/Divergence

In [1]:
!pip install pandas --upgrade --quiet
!pip install numpy --upgrade --quiet
!pip install yfinance --upgrade --quiet
!pip install ta --upgrade --quiet
!pip install pygooglenews --upgrade --quiet
!pip install matplotlib --upgrade --quiet

     |████████████████████████████████| 12.2 MB 5.3 MB/s 
     |████████████████████████████████| 17.1 MB 4.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.5 which is incompatible.
     |████████████████████████████████| 62 kB 993 kB/s 
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 46.3 MB/s 
     |████████████████████████████████| 128 kB 45.1 MB/s 
     |████████████████████████████████| 9.4 MB 5.2 MB/s 
     |████████████████████████████████| 965 kB 58.3 MB/s 
     |████████████████████████████████| 295 kB 57.7 MB/s 


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import ta

def get_stocks(name):
  #time
  current_time = datetime.datetime.now()
  start_date = '2014-01-01'
  end_date = current_time

  # Set the ticker
  ticker = name 
  # Get the data
  data = yf.download(ticker, start_date, end_date)
  # Print the last 5 rows
  #print(data.head())
  df = pd.DataFrame(data=data)
  return df

In [3]:
#0023.HK - BEA
#0005.HK - HSBC
#^HSI - Heng Seng Index
df_stock_hsbc = get_stocks('0005.HK')
df_stock_bea = get_stocks('0023.HK')
df_stock_hsi = get_stocks('^HSI')

'''
df_stock_hsbc.to_excel("Stock HSBC.xlsx", index = None)
df_stock_bea.to_excel("Stock BEA.xlsx", index = None)
df_stock_hsi.to_excel("Stock HSI.xlsx", index = None)
'''

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


'\ndf_stock_hsbc.to_excel("Stock HSBC.xlsx", index = None)\ndf_stock_bea.to_excel("Stock BEA.xlsx", index = None)\ndf_stock_hsi.to_excel("Stock HSI.xlsx", index = None)\n'

#TA

In [4]:
df_stock_hsbc.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2203 entries, 2014-01-02 to 2022-12-02
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       2203 non-null   float64
 1   High       2203 non-null   float64
 2   Low        2203 non-null   float64
 3   Close      2203 non-null   float64
 4   Adj Close  2203 non-null   float64
 5   Volume     2203 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 120.5 KB


https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#trend-indicators

In [17]:
def df_ta(df):
  df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
  df['MFI'] = ta.volume.MFIIndicator(df['High'],df['Low'],df['Close'],df['Volume'], window=14).money_flow_index()
  df['MACD'] = ta.trend.MACD(df['Close'],window_slow = 26, window_fast = 12, window_sign = 9).macd()
  df["Returns"] = np.log(df['Close'] / df['Close'].shift())
  
  temp = []
  col_list = df["Returns"].values.tolist()
  for i in col_list:
    if i >= 0:
      temp.append(0)
    elif i < 0:
      temp.append(1)
    else:
      temp.append(None)

  df["Sorted_Return"] = temp

  new_df = df.dropna()
  return new_df

In [18]:
new_df_stock_hsbc = df_ta(df_stock_hsbc) 
new_df_stock_bea = df_ta(df_stock_bea)
new_df_stock_hsi = df_ta(df_stock_hsi)

In [19]:
new_df_stock_hsbc

,Open,High,Low,Close,Adj Close,Volume,RSI,MFI,MACD,Returns,Sorted_Return
Date,,,,,,,,,,,
2014-02-10,79.849998,79.849998,79.150002,79.199997,51.115627,18100317,24.119626,24.535382,-1.550945,-0.001262,1.0
2014-02-11,79.050003,79.800003,79.050003,79.699997,51.438339,18579742,29.496018,26.116428,-1.538075,0.006293,0.0
2014-02-12,80.699997,81.300003,80.599998,81.000000,52.277370,23042878,41.167767,32.736868,-1.406760,0.016180,0.0
2014-02-13,82.000000,82.000000,81.199997,81.349998,52.503250,17349305,43.862299,33.880771,-1.259926,0.004312,0.0
2014-02-14,82.449997,82.500000,81.900002,82.099998,52.987309,16175910,49.228509,39.749862,-1.070699,0.009177,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-28,45.950001,45.950001,44.950001,45.599998,45.599998,16162878,63.938026,67.511102,0.931755,-0.008734,1.0
2022-11-29,45.700001,46.250000,45.599998,46.250000,46.250000,21580355,67.097299,74.147243,0.988716,0.014154,0.0
2022-11-30,47.400002,47.650002,46.900002,47.250000,47.250000,27847858,71.267704,80.291812,1.101848,0.021391,0.0


In [20]:
new_df_stock_hsbc.to_excel("RSI Sorted.xlsx")

#Do Cross Correlation
- 

In [8]:
'''
from datetime 
a = 'Sun, 28 Mar 2021 07:00:00 GMT'
b = a[5:16]
b = b.split(' ')
year = b[2]
month = b[1]
day = b[0]
month_dict = {'Jan' : '01',
        'Feb' : '02',
        'Mar' : '03',
        'Apr' : '04',
        'May' : '05',
        'Jun' : '06',
        'Jul' : '07',
        'Aug' : '08',
        'Sep' : '09',
        'Oct' : '10',
        'Nov' : '11',
        'Dec' : '12',
        }

for i in month_dict:
  if month == i:
    month = i.values()

resutl = year+'/'+month+'/'+day
'''

"\nfrom datetime \na = 'Sun, 28 Mar 2021 07:00:00 GMT'\nb = a[5:16]\nb = b.split(' ')\nyear = b[2]\nmonth = b[1]\nday = b[0]\nmonth_dict = {'Jan' : '01',\n        'Feb' : '02',\n        'Mar' : '03',\n        'Apr' : '04',\n        'May' : '05',\n        'Jun' : '06',\n        'Jul' : '07',\n        'Aug' : '08',\n        'Sep' : '09',\n        'Oct' : '10',\n        'Nov' : '11',\n        'Dec' : '12',\n        }\n\nfor i in month_dict:\n  if month == i:\n    month = i.values()\n\nresutl = year+'/'+month+'/'+day\n"